# 分布式训练模型

In [1]:
import pandas as pd
import numpy as np
from pandas import DataFrame
from datetime import timedelta
from numpy import ndarray
from typing import Union, List, Dict
from sklearn.preprocessing import MinMaxScaler
import os
import re
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import torch.nn.functional as F
from tqdm import tqdm  # 打印进度条
import math
import matplotlib.pyplot as plt
import seaborn as sns
from typing import List
from pandas.tseries import offsets
from pandas.tseries.frequencies import to_offset
from sklearn.metrics import r2_score, mean_squared_error
import joblib
import warnings

warnings.filterwarnings("ignore")
plt.rcParams['font.sans-serif'] = ['SimHei']  # 显示中文标签
plt.rcParams['axes.unicode_minus'] = False

深度学习的分布式训练通常用于加速训练过程，特别是在处理大规模数据集时，或者需要大量计算资源的任务。PyTorch 提供了多种方法来进行分布式训练，包括数据并行、模型并行和混合并行等。

## 数据并行

数据并行是最常见的分布式训练方式。它的基本思想是将数据划分成多个小批次，并将每个批次分配给不同的计算节点（如 GPU）。每个节点计算梯度，并将梯度进行汇总更新。

### 单机多卡

torch.nn.DataParallel 是 PyTorch 提供的一个简单的工具，用于在多个 GPU 上进行数据并行训练。

In [2]:
# 使用DataParallel将模型并行化
model = nn.DataParallel(model)

NameError: name 'model' is not defined

### 多机多卡

torch.nn.parallel.DistributedDataParallel（DDP）是 PyTorch 中推荐的分布式训练方式，相比 DataParallel，DDP 在性能上有显著的提升，特别是在多机多卡的环境下。

## 模型并行

当模型过大，无法完全放入单个 GPU 时，可以使用模型并行。在这种方式下，模型的不同部分会分配到不同的设备（GPU）上，每个设备处理它自己的部分。

例如，如果你有一个非常大的模型，可以将其分成多个部分，每个部分运行在不同的 GPU 上。PyTorch 允许通过将模型的不同层放到不同的设备上来实现模型并行。

## 混合并行

混合并行结合了数据并行和模型并行。它适用于非常大的模型或数据集，可以在多个节点和多个 GPU 上分配任务。